In [1]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100]))

In [6]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

In [7]:
models = [
    (
        "Logistic Regression", 
        {"C": 1, "solver": 'liblinear'},
        LogisticRegression(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    )
]

In [8]:
reports = []

for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [3]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

In [5]:
# dagshub setup

import dagshub
dagshub.init(repo_owner='saad946', repo_name='mlflow-dagshub-implementation-demo', mlflow=True)

Initialized MLflow to track repo "saad946/mlflow-dagshub-implementation-demo"

Repository saad946/mlflow-dagshub-implementation-demo initialized!

In [ ]:
# Ideally you will not require following 4 lines if you have started fresh and do not have any previous dagshub credentials on your computer
import os
os.environ['MLFLOW_TRACKING_USERNAME'] = 'saad946' # 'learnpythonlanguage'
os.environ['MLFLOW_TRACKING_PASSWORD'] = '2f4f141bc42dfdghjkkl8b043627bc56566' # 
os.environ['MLFLOW_TRACKING_URI'] = 'https://dagshub.com/saad946/mlflow-dagshub-implementation-demo.mlflow' # https://dagshub.com/learnpythonlanguage/mlflow_dagshub_demo.mlflow

# Initialize MLflow
mlflow.set_experiment("Anomaly Detection with Registry")
# mlflow.set_tracking_uri("http://localhost:5000")

for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_params(params)
        mlflow.log_metrics({
            'accuracy': report['accuracy'],
            'recall_class_1': report['1']['recall'],
            'recall_class_0': report['0']['recall'],
            'f1_score_macro': report['macro avg']['f1-score']
        })  
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")  